In [5]:
import pandas as pd
import random
import math

# Function to convert string of space-separated numbers to a list of integers
def parse_list(string):
    return list(map(int, string.strip("[]").split()))

# Function to calculate the total value of the current state
def calculate_total_value(weights, values, state, capacity):
    total_weight = sum(weights[i] for i in range(len(state)) if state[i] == 1)
    total_value = sum(values[i] for i in range(len(state)) if state[i] == 1)
    if total_weight > capacity:
        return 0  # Penalize if the total weight exceeds the capacity
    else:
        return total_value

# Simulated Annealing algorithm
def simulated_annealing(weights, values, capacity, initial_state, temp, cooling_rate, stopping_temp):
    current_state = initial_state
    current_value = calculate_total_value(weights, values, current_state, capacity)
    best_state = list(current_state)
    best_value = current_value
    
    while temp > stopping_temp:
        neighbor = list(current_state)
        # Flip a random bit to generate a neighbor state
        bit = random.randint(0, len(neighbor) - 1)
        neighbor[bit] = 1 - neighbor[bit]
        
        neighbor_value = calculate_total_value(weights, values, neighbor, capacity)
        
        # Calculate the acceptance probability
        delta = neighbor_value - current_value
        if delta > 0 or random.random() < math.exp(delta / temp):
            current_state = neighbor
            current_value = neighbor_value
            
            # Update the best found state if the neighbor is better
            if neighbor_value > best_value:
                best_state = neighbor
                best_value = neighbor_value
        
        # Cool down the temperature
        temp *= (1 - cooling_rate)
    
    return best_state, best_value

# Load the dataset
df = pd.read_csv('/mnt/d/Algo code/knapsack_5_items.csv')

# Initialize the overall best value and corresponding state and row index
overall_best_value = 0
overall_best_state = []
overall_best_row_index = -1

# Process each row in the dataset
for index, row in df.iterrows():
    weights = parse_list(row['Weights'])
    values = parse_list(row['Prices'])
    capacity = int(row['Capacity'])
    
    # Initial state (solution) is all zeros
    initial_state = [0] * len(weights)
    
    # Parameters for the SA algorithm
    initial_temperature = 10000
    cooling_rate = 0.003
    stopping_temperature = 1

    # Apply Simulated Annealing
    best_state, best_value = simulated_annealing(
        weights, values, capacity, initial_state, initial_temperature, cooling_rate, stopping_temperature
    )
    
    # Update overall best value, state, and row index if the current one is better
    if best_value > overall_best_value:
        overall_best_value = best_value
        overall_best_state = best_state
        overall_best_row_index = index

# Print the best value found along with the row index and state
print(f"Row: {overall_best_row_index}, Best state: {overall_best_state}, Best value: {overall_best_value}")

Row: 7229, Best state: [1, 1, 1, 1, 1], Best value: 82


In [1]:
pwd


'/mnt/d/Algo code'

In [6]:
import pandas as pd
import random
import math
from concurrent.futures import ThreadPoolExecutor

def parse_list(string):
    return list(map(int, string.strip("[]").split()))

def calculate_total_value(weights, values, state, capacity):
    total_weight = total_value = 0
    for i in range(len(state)):
        if state[i] == 1:
            total_weight += weights[i]
            total_value += values[i]
    if total_weight > capacity:
        return 0
    return total_value

def simulated_annealing(weights, values, capacity, initial_state, temp, cooling_rate, stopping_temp):
    current_state = initial_state[:]
    current_value = calculate_total_value(weights, values, current_state, capacity)
    best_state = current_state[:]
    best_value = current_value
    
    while temp > stopping_temp:
        bit = random.randint(0, len(current_state) - 1)
        neighbor = current_state[:]
        neighbor[bit] = 1 - neighbor[bit]
        
        neighbor_value = calculate_total_value(weights, values, neighbor, capacity)
        delta = neighbor_value - current_value
        
        if delta > 0 or random.random() < math.exp(delta / temp):
            current_state = neighbor
            current_value = neighbor_value
            
            if neighbor_value > best_value:
                best_state = neighbor
                best_value = neighbor_value
        
        temp *= (1 - cooling_rate)
    
    return best_state, best_value

def process_row(row):
    weights = parse_list(row['Weights'])
    values = parse_list(row['Prices'])
    capacity = int(row['Capacity'])
    
    initial_state = [0] * len(weights)
    
    initial_temperature = 10000
    cooling_rate = 0.003
    stopping_temperature = 1
    
    return simulated_annealing(weights, values, capacity, initial_state, initial_temperature, cooling_rate, stopping_temperature)

df = pd.read_csv('/mnt/d/Algo code/knapsack_5_items.csv')

overall_best_value = 0
overall_best_state = []
overall_best_row_index = -1

with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_row, [row for _, row in df.iterrows()]))

for index, (best_state, best_value) in enumerate(results):
    if best_value > overall_best_value:
        overall_best_value = best_value
        overall_best_state = best_state
        overall_best_row_index = index

print(f"Row: {overall_best_row_index}, Best state: {overall_best_state}, Best value: {overall_best_value}")


Row: 7229, Best state: [1, 1, 1, 1, 1], Best value: 82
